In [1]:
# Combine_zips_with_any_intersection
import pandas as pd
import numpy as np
import datetime
import json
from haversine import haversine
import zipcodes
import geocoder
Bing_key = "Avga1gIvFuPIXtcrunxy_sYkQkrLSKTb9XKFrUAb6nnfzG4eC6SwuuznZAw-byWJ"

# https://www.unitedstateszipcodes.org/

In [2]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json"))

In [3]:
haversine(zip_centers['78734'],[30.34570611,-97.96602215],miles=True)

2.5389565450652434

In [3]:
'''
data=pd.read_csv("/home/jian/Projects/Smoothie_King/TA/data_from_Jay/zip_within_3_mile.csv",dtype=str)
data['zip_code']=data['zip_code'].apply(lambda x: x.zfill(5))
'''

'\ndata=pd.read_csv("/home/jian/Projects/Smoothie_King/TA/data_from_Jay/zip_within_3_mile.csv",dtype=str)\ndata[\'zip_code\']=data[\'zip_code\'].apply(lambda x: x.zfill(5))\n'

In [4]:
store_list=pd.read_excel("/home/jian/Projects/Smoothie_King/TA/StoreList.xlsx",dtype=str)
closed_stores=store_list[store_list['status']=="Closed"].reset_index()

store_list=store_list[store_list['status']!="Closed"].reset_index()
del store_list['index']
store_list['zip']=store_list['zip'].apply(lambda x: x.zfill(5))
store_list['storenumber']=store_list['storenumber'].astype(int)
store_list=store_list.sort_values(['storenumber'])
store_list['storenumber']=store_list['storenumber'].astype(str)
store_list=store_list.reset_index()
del store_list['index']

In [5]:
store_list['Bing_LatLng']=np.nan
store_list['Bing_Address']=np.nan
for i in range(len(store_list)):
    if store_list['Latitude'][i]=="nan":
        search_key=store_list['Address'][i]+", "+store_list['city'][i]+", "+store_list['state'][i]+", "+store_list['zip'][i]
        
        response=geocoder.bing(search_key,key=Bing_key)
        if len(response)>0:
            lat_lng=response.latlng
            store_list['Latitude'][i]=lat_lng[0]
            store_list['Longitude'][i]=lat_lng[1]
            store_list['Bing_LatLng'][i]="Bing"
            store_list['Bing_Address'][i]=response.address


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/si

In [6]:
store_list['Zip_of_LatLng']=np.nan
store_list['Zip_of_Address']=np.nan

# Manually change of the lat/long for store 1670
store_list['Latitude'][store_list['Latitude']=="nan"]=39.9353979
store_list['Longitude'][store_list['Longitude']=="nan"]=-91.3444578

for i in range(len(store_list)):

    coor_pair=[float(store_list['Latitude'][i]),float(store_list['Longitude'][i])]
    response=geocoder.bing(coor_pair,key=Bing_key,method='reverse')
    postal_reverse_latlng=response.postal
    store_list['Zip_of_LatLng'][i]=postal_reverse_latlng

    search_key=store_list['Address'][i]+", "+store_list['city'][i]+", "+store_list['state'][i]+", "+store_list['zip'][i]
    response=geocoder.bing(search_key,key=Bing_key)
    postal_reverse_add=response.postal
    store_list['Zip_of_Address'][i]=postal_reverse_add
    
store_list['Zip_of_LatLng']=store_list['Zip_of_LatLng'].apply(lambda x: str(x).split(".")[0].zfill(5))
store_list['Zip_of_Address']=store_list['Zip_of_Address'].apply(lambda x: str(x).split(".")[0].zfill(5))
store_list['store_zip']=store_list['zip'].apply(lambda x: x.split("-")[0].zfill(5))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [7]:
check=store_list[(store_list['Zip_of_LatLng']!=store_list['store_zip']) &\
                 (store_list['Zip_of_Address']!=store_list['store_zip']) &\
                 (store_list['zip']!='00nan') &\
                 ((store_list['Zip_of_LatLng']!='00nan') | (store_list['Zip_of_Address']!='00nan'))]


In [8]:
manually_revise_zip={"402":"36832","595":"75201","953":"27707","1279":"70808","58":"33618","387":"77478",
                     "1464":"07901","1252":"73013","1282":"70803","1386":"62711","989":"70130","1132":"31605",
                     "1384":"30075","1432":"30281","1125":"33136"}
manually_revise_zip.keys()

dict_keys(['402', '595', '953', '1279', '58', '387', '1464', '1252', '1282', '1386', '989', '1132', '1384', '1432', '1125'])

In [9]:
store_list=store_list.reset_index()
del store_list['index']
store_list['revised_store_zip']=np.nan
for i in range(len(store_list)):
    if (store_list['storenumber'][i] in manually_revise_zip.keys()):
        store_list['revised_store_zip'][i]=manually_revise_zip[store_list['storenumber'][i]]
    elif ((store_list['Zip_of_LatLng'][i]=="00nan") & (store_list['Zip_of_Address'][i]=="00nan")):
        store_list['revised_store_zip'][i]=store_list['store_zip'][i]        
    elif ((store_list['store_zip'][i]==store_list['Zip_of_LatLng'][i]) |\
          (store_list['store_zip'][i]==store_list['Zip_of_Address'][i])):
        store_list['revised_store_zip'][i]=store_list['store_zip'][i]
    elif (store_list['Zip_of_LatLng'][i]==store_list['Zip_of_Address'][i]):
        store_list['revised_store_zip'][i]=store_list['Zip_of_LatLng'][i]
store_list['revised_store_zip']=store_list['revised_store_zip'].apply(lambda x: str(int(x)).zfill(5))      

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarn

In [10]:
store_list[pd.isnull(store_list['revised_store_zip'])]

,storenumber,Address,city,state,zip,StoreOpenDate,Latitude,Longitude,Ownership,Franchisee_Name,IPAddress,DriveThru,status,StoreCloseDate,Bing_LatLng,Bing_Address,Zip_of_LatLng,Zip_of_Address,store_zip,revised_store_zip


In [11]:
store_list[store_list['revised_store_zip']=="00nan"]

,storenumber,Address,city,state,zip,StoreOpenDate,Latitude,Longitude,Ownership,Franchisee_Name,IPAddress,DriveThru,status,StoreCloseDate,Bing_LatLng,Bing_Address,Zip_of_LatLng,Zip_of_Address,store_zip,revised_store_zip


In [12]:
overwrite_zip_stores=store_list[store_list['revised_store_zip']!=store_list['store_zip']]
overwrite_zip_stores.shape

(56, 20)

# Get 3 miles zips based on lat lng

In [13]:
store_list['zips_3_miles']=np.nan
for i in range(len(store_list)):
    store_center=[float(store_list['Latitude'][i]),float(store_list['Longitude'][i])]
    list_zip_3_miles=[store_list['revised_store_zip'][i]]
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=3:
            list_zip_3_miles=list_zip_3_miles+[zip_cd]
    store_list['zips_3_miles'][i]=list(set(list_zip_3_miles))
            

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
data=store_list.copy()
store_list_df=store_list.copy()
data.columns.tolist()
del data['Bing_LatLng']
del data['Bing_Address']
del data['Zip_of_LatLng']
del data['Zip_of_Address']
del data['store_zip']

In [15]:
type(store_list)

pandas.core.frame.DataFrame

In [16]:
data['TA']=np.nan
data=data.reset_index()
del data['index']
df_TA_zips=pd.DataFrame({"store":[data['storenumber'][0]]*len(data['zips_3_miles'][0]),"zip":data['zips_3_miles'][0],"TA":[1]*len(data['zips_3_miles'][0])},index=[1]*len(data['zips_3_miles'][0]))



In [17]:
TA_counter=1

for i in range(1,len(data)):
    intersection_zip=list(set(data['zips_3_miles'][i]).intersection(set(df_TA_zips['zip'].unique().tolist())))
    if len(intersection_zip)==0:
        TA_counter+=1
        df_TA_zips=df_TA_zips.append(pd.DataFrame({"store":[data['storenumber'][i]]*len(data['zips_3_miles'][i]),"zip":data['zips_3_miles'][i],"TA":[TA_counter]*len(data['zips_3_miles'][i])},index=[i]*len(data['zips_3_miles'][i]))).drop_duplicates()
        
    else:
        df_intersection=df_TA_zips[df_TA_zips['zip'].isin(intersection_zip)]
        group_df_intersection=df_intersection.groupby(['TA'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False)
        selected_TA=group_df_intersection['TA'][0] 
        
        df_TA_zips_0=df_TA_zips[~df_TA_zips['TA'].isin(set(group_df_intersection['TA']))]
        df_TA_zips_1=df_TA_zips[df_TA_zips['TA'].isin(group_df_intersection['TA'].tolist())]
        df_TA_zips_1['TA']=selected_TA
        df_TA_zips=df_TA_zips_0.append(df_TA_zips_1).append(pd.DataFrame({"store":[data['storenumber'][i]]*len(data['zips_3_miles'][i]),"zip":data['zips_3_miles'][i],"TA":[selected_TA]*len(data['zips_3_miles'][i])},index=[i]*len(data['zips_3_miles'][i]))).drop_duplicates()
        
dict_TA_zips=df_TA_zips.set_index('zip').to_dict()['TA']
dict_TA_store=df_TA_zips.set_index('store').to_dict()['TA']
data['TA']=data['storenumber'].apply(lambda x: dict_TA_store[x])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [18]:
data.shape

(886, 17)

In [19]:
data['TA'].max()

530

In [20]:
df_ta_num_unique=data[['TA']].drop_duplicates().reset_index()
del df_ta_num_unique['index']
df_ta_num_unique['new_TA']=[x+1 for x in range(len(df_ta_num_unique))]

dict_ta_num_unique=df_ta_num_unique.set_index(['TA']).to_dict()['new_TA']
data['TA']=data['TA'].apply(lambda x: dict_ta_num_unique[x])
df_TA_zips['TA']=df_TA_zips['TA'].apply(lambda x: dict_ta_num_unique[x])


In [21]:

summary_store=data.groupby("TA")['storenumber'].count().to_frame().reset_index().rename(columns={"storenumber":"store_count"})
summary_zip=df_TA_zips[['TA','zip']].drop_duplicates().groupby("TA")['zip'].count().to_frame().reset_index().rename(columns={"zip":"zip_count"})
# summary_store_2=df_TA_zips[['TA','store']].drop_duplicates().groupby("TA")['store'].count().to_frame().reset_index().rename(columns={"store":"store_count_2"})
summary_store_list=data.groupby("TA")['storenumber'].apply(list).to_frame().reset_index().rename(columns={"storenumber":"store_list"})
summary_zip_list=df_TA_zips[['TA','zip']].drop_duplicates().groupby("TA")['zip'].apply(list).to_frame().reset_index().rename(columns={"zip":"zip_list"})



summary_by_TA=pd.merge(summary_store,summary_zip,on="TA",how="outer")
summary_by_TA=pd.merge(summary_by_TA,summary_store_list,on="TA",how="outer")
summary_by_TA=pd.merge(summary_by_TA,summary_zip_list,on="TA",how="outer")

# summary=pd.merge(summary,summary_store_2,on="TA",how="outer")
TA_Store_zip_list=data.groupby(['TA'])['revised_store_zip'].apply(set).to_frame().reset_index().rename(columns={"revised_store_zip":"store_zip_list"})
summary_by_TA=pd.merge(summary_by_TA,TA_Store_zip_list,on="TA",how="left")


In [ ]:
summary_by_TA.hea

In [22]:
summary_by_store_count=summary_by_TA.groupby(['store_count'])['TA'].count().to_frame().reset_index().rename(columns={"TA":"TA_count"})
summary_by_store_list=summary_by_TA.groupby(['store_count'])['TA'].apply(list).to_frame().reset_index().rename(columns={"TA":"TA_list"})
summary_by_store_count=pd.merge(summary_by_store_count,summary_by_store_list,on="store_count",how="outer")




In [23]:
df_zip_dist_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
del df_zip_dist_in_TA['index']

output_distance_zip_in_TA=pd.DataFrame()
counter_k=0
for ta,group in df_zip_dist_in_TA.groupby(['TA']):
    group=group.reset_index()
    del group['index']
    
    if len(group)>1:
    
        dist_list=[]

        for i in range(len(group)):
            zip_hold=group['zip'][i]

            if zip_hold not in zip_centers.keys():
                try:
                    zip_hold_center=(zipcodes.matching(zip_hold)[0]['lat'],zipcodes.matching(zip_hold)[0]['long'])
                except:
                    print("zip not found, ",zip_hold)

            else:
                zip_hold_center=zip_centers[zip_hold]

            for j in range(i+1,len(group)):
                zip_var=group['zip'][j]
                if zip_var not in zip_centers.keys():
                    try:
                        zip_var_center=(zipcodes.matching(zip_var)[0]['lat'],zipcodes.matching(zip_var)[0]['long'])
                    except:
                        print("zip not found, ",zip_hold)

                else:
                    zip_var_center=zip_centers[zip_var]

                try:
                    dist=haversine(zip_hold_center,zip_var_center,miles=True)
                except:
                    dist=np.nan

                dist_list=dist_list+[dist]
        df=pd.DataFrame({"TA":ta,"dist_min":min(dist_list),"dist_max":max(dist_list),"dist_median":np.median(dist_list),"All_dist":[dist_list]},index=[counter_k])
        counter_k+=1
        output_distance_zip_in_TA=output_distance_zip_in_TA.append(df)
    else:
        df=pd.DataFrame({"TA":ta,"dist_min":0,"dist_max":0,"dist_median":0,"All_dist":"single_zip"},index=[counter_k])
        output_distance_zip_in_TA=output_distance_zip_in_TA.append(df)
    

In [24]:
zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
zip_DMA=zip_DMA.iloc[:,[0,2]]
zip_DMA.columns=['zip','DMA']

In [25]:
df_city_state_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
del df_city_state_in_TA['index']

def city_of_zip(x):
    try:
        city=zipcodes.matching(x)[0]['city']
    except:
        city=np.nan
    return city

def state_of_zip(x):
    try:
        state=zipcodes.matching(x)[0]['state']
    except:
        state=np.nan
    return state
    
df_city_state_in_TA['city']=df_city_state_in_TA['zip'].apply(lambda x: city_of_zip(x))
df_city_state_in_TA['state']=df_city_state_in_TA['zip'].apply(lambda x: state_of_zip(x))
df_city_state_in_TA['city']=df_city_state_in_TA['city']+" ("+df_city_state_in_TA['state']+")"



In [26]:
df_DMA_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
del df_DMA_in_TA['index']
df_DMA_in_TA=pd.merge(df_DMA_in_TA,zip_DMA,on="zip",how="left")
df_DMA_in_TA.head(2)

,TA,zip,DMA
0,3,70047,NEW ORLEANS
1,3,70031,NEW ORLEANS


In [27]:
df_city_TA_list=df_city_state_in_TA.groupby(['TA'])['city'].apply(set).to_frame().reset_index()
df_city_TA_list=df_city_TA_list.rename(columns={"city":"city_list"})
df_state_TA_list=df_city_state_in_TA.groupby(['TA'])['state'].apply(set).to_frame().reset_index()
df_state_TA_list=df_state_TA_list.rename(columns={"state":"state_list"})
df_DMA_TA_list=df_DMA_in_TA.groupby(['TA'])['DMA'].apply(set).to_frame().reset_index()
df_DMA_TA_list=df_DMA_TA_list.rename(columns={"DMA":"DMA_list"})


In [28]:
manually_city_dict_of_zip={"29486":"Summerville"}
manually_state_dict_of_zip={"29486":"SC"}
df_city_state_in_TA['city'][df_city_state_in_TA['zip']=='29486']=manually_city_dict_of_zip['29486']
df_city_state_in_TA['state'][df_city_state_in_TA['zip']=='29486']=manually_state_dict_of_zip['29486']

removed_2_zips_df=df_city_state_in_TA[pd.isnull(df_city_state_in_TA['city'])]
df_city_state_in_TA=df_city_state_in_TA[~pd.isnull(df_city_state_in_TA['city'])]
# zip "08644","76190" removed to determine cities/states because of not existing as in https://www.unitedstateszipcodes.org/ 

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [29]:
counter_k=1
df_primary_city_state=pd.DataFrame()
df_primary_DMA=pd.DataFrame()
for ta,group in df_city_state_in_TA.groupby(['TA']):
    df_city=group.groupby(['city'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
    del df_city['index']
    primary_city=df_city['city'][0]
    
    df_state=group.groupby(['state'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
    del df_state['index']
    primary_state=df_state['state'][0]
    
    df=pd.DataFrame({"TA":ta,"Primary_City":primary_city,"Primary_State":primary_state},index=[counter_k])
    counter_k+=1
    df_primary_city_state=df_primary_city_state.append(df)

    
for ta,group in df_DMA_in_TA.groupby(['TA']):
    df_DMA=group.groupby(['DMA'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
    del df_DMA['index']
    primary_DMA=df_DMA['DMA'][0]
    
    df=pd.DataFrame({"TA":ta,"Primary_DMA":primary_DMA},index=[counter_k])
    counter_k+=1
    df_primary_DMA=df_primary_DMA.append(df)

In [30]:
summary_by_TA=pd.merge(summary_by_TA,df_city_TA_list,on="TA",how="left")
summary_by_TA=pd.merge(summary_by_TA,df_state_TA_list,on="TA",how="left")
summary_by_TA=pd.merge(summary_by_TA,df_DMA_TA_list,on="TA",how="left")

summary_by_TA=pd.merge(summary_by_TA,df_primary_city_state,on="TA",how="left")
summary_by_TA=pd.merge(summary_by_TA,df_primary_DMA,on="TA",how="left")

summary_by_TA=pd.merge(summary_by_TA,output_distance_zip_in_TA,on="TA",how="left")

In [31]:
data['Latitude']=data['Latitude'].astype(float)
data['Longitude']=data['Longitude'].astype(float)
data['TA']=data['TA'].astype(str)
summary_by_TA['TA']=summary_by_TA['TA'].astype(str)

# Revise TA

In [32]:
summary_by_TA['ratio_max_media']=summary_by_TA['dist_max']/summary_by_TA['dist_median']

summary_by_TA_to_revise=summary_by_TA[(summary_by_TA['ratio_max_media']>2) &\
                                      (summary_by_TA['store_count']>=2) &\
                                     (summary_by_TA['dist_max']>12)]
summary_by_TA_to_keep=summary_by_TA[(summary_by_TA['ratio_max_media']<=2) |\
                                      (summary_by_TA['store_count']<2) |\
                                     (summary_by_TA['dist_max']<=12)]
summary_by_TA_to_revise=summary_by_TA_to_revise.reset_index()
del summary_by_TA_to_revise['index']

summary_by_TA_to_keep=summary_by_TA_to_keep.reset_index()
del summary_by_TA_to_keep['index']

In [33]:
len(summary_by_TA['TA'].unique())

493

In [34]:
summary_by_TA['TA'].apply(lambda x: type(x)).unique()

array([<class 'str'>], dtype=object)

In [35]:
summary_by_TA['TA'].astype(int).max()

493

In [36]:
loop_counter=0
while len(summary_by_TA_to_revise)>0:
    loop_counter+=1
    if loop_counter>30:
        break
    
    summary_by_TA_to_revise=summary_by_TA_to_revise.reset_index()
    del summary_by_TA_to_revise['index']

    summary_by_TA_to_keep=summary_by_TA_to_keep.reset_index()
    del summary_by_TA_to_keep['index']

    data_keep=data[data['TA'].isin(summary_by_TA_to_keep['TA'])]
    data_revise=data[~data['TA'].isin(summary_by_TA_to_keep['TA'])]
    data_revise=data_revise.reset_index()
    del data_revise['index']


    store_lat_dict=data_revise.set_index(['storenumber']).to_dict()['Latitude']
    store_lng_dict=data_revise.set_index(['storenumber']).to_dict()['Longitude']


    store_sub_df=pd.DataFrame()
    for i in range(len(summary_by_TA_to_revise)):
        TA=summary_by_TA_to_revise['TA'][i]
        store_list=summary_by_TA_to_revise['store_list'][i].copy()
        initial_dist=0
        store_pair=[np.nan,np.nan]
        while len(store_list)>=2:
            store_hold=store_list[0]
            store_list.remove(store_hold)
            store_hold_center=[store_lat_dict[store_hold],store_lng_dict[store_hold]]
            for store_var in store_list:
                store_var_center=[store_lat_dict[store_var],store_lng_dict[store_var]]
                dist=haversine(store_hold_center,store_var_center,miles=True)
                if dist>initial_dist:
                    initial_dist=dist
                    store_pair=[store_hold,store_var]
        store_a=store_pair[0]
        store_b=store_pair[1]

        store_a_center=[store_lat_dict[store_a],store_lng_dict[store_a]]
        store_b_center=[store_lat_dict[store_b],store_lng_dict[store_b]]
        store_list=summary_by_TA_to_revise['store_list'][i].copy()
        for store in store_list:
            store_center=[store_lat_dict[store],store_lng_dict[store]]
            dist_a=haversine(store_a_center,store_center,miles=True)
            dist_b=haversine(store_b_center,store_center,miles=True)
            if dist_a<dist_b:
                sub_group="a"
            else:
                sub_group="b"
            df=pd.DataFrame({"storenumber":store,"TA":TA,"sub_group":sub_group},index=[store])
            store_sub_df=store_sub_df.append(df)
    
    store_sub_df['TA']=store_sub_df['TA'].astype(str)
    store_sub_df['TA']=store_sub_df['TA']+"_"+store_sub_df['sub_group']
    store_sub_df=store_sub_df[['storenumber','TA']]
    store_sub_df_dic=store_sub_df.set_index(['storenumber']).to_dict()['TA']
                    
    data_revise['TA']=data_revise['storenumber'].apply(lambda x: store_sub_df_dic[x])
    data=data_keep.append(data_revise)
    data=data.sort_values(['storenumber'])
    data=data.reset_index()
    del data['index']
    data['TA']=data['TA'].apply(lambda x: str(x).zfill(5))

    df_TA_zips=pd.DataFrame()
    for i in range(len(data)):
        df=pd.DataFrame({"store":[data['storenumber'][i]]*len(data['zips_3_miles'][i]),
                         "TA":[data['TA'][i]]*len(data['zips_3_miles'][i]),
                         "zip":data['zips_3_miles'][i]},index=data['zips_3_miles'][i])
        df_TA_zips=df_TA_zips.append(df)


    summary_store=data.groupby("TA")['storenumber'].count().to_frame().reset_index().rename(columns={"storenumber":"store_count"})
    summary_zip=df_TA_zips[['TA','zip']].drop_duplicates().groupby("TA")['zip'].count().to_frame().reset_index().rename(columns={"zip":"zip_count"})
    summary_store_list=data.groupby("TA")['storenumber'].apply(list).to_frame().reset_index().rename(columns={"storenumber":"store_list"})
    summary_zip_list=df_TA_zips[['TA','zip']].drop_duplicates().groupby("TA")['zip'].apply(list).to_frame().reset_index().rename(columns={"zip":"zip_list"})

    summary_by_TA=pd.merge(summary_store,summary_zip,on="TA",how="outer")
    summary_by_TA=pd.merge(summary_by_TA,summary_store_list,on="TA",how="outer")
    summary_by_TA=pd.merge(summary_by_TA,summary_zip_list,on="TA",how="outer")

    # summary=pd.merge(summary,summary_store_2,on="TA",how="outer")
    TA_Store_zip_list=data.groupby(['TA'])['revised_store_zip'].apply(set).to_frame().reset_index().rename(columns={"zip_code":"store_zip_list"})
    summary_by_TA=pd.merge(summary_by_TA,TA_Store_zip_list,on="TA",how="left")


    summary_by_store_count=summary_by_TA.groupby(['store_count'])['TA'].count().to_frame().reset_index().rename(columns={"TA":"TA_count"})
    summary_by_store_list=summary_by_TA.groupby(['store_count'])['TA'].apply(list).to_frame().reset_index().rename(columns={"TA":"TA_list"})
    summary_by_store_count=pd.merge(summary_by_store_count,summary_by_store_list,on="store_count",how="outer")

    df_zip_dist_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
    del df_zip_dist_in_TA['index']

    output_distance_zip_in_TA=pd.DataFrame()
    counter_k=0
    for ta,group in df_zip_dist_in_TA.groupby(['TA']):
        group=group.reset_index()
        del group['index']

        if len(group)>1:

            dist_list=[]

            for i in range(len(group)):
                zip_hold=group['zip'][i]

                if zip_hold not in zip_centers.keys():
                    try:
                        zip_hold_center=(zipcodes.matching(zip_hold)[0]['lat'],zipcodes.matching(zip_hold)[0]['long'])
                    except:
                        print("zip not found, ",zip_hold)

                else:
                    zip_hold_center=zip_centers[zip_hold]

                for j in range(i+1,len(group)):
                    zip_var=group['zip'][j]
                    if zip_var not in zip_centers.keys():
                        try:
                            zip_var_center=(zipcodes.matching(zip_var)[0]['lat'],zipcodes.matching(zip_var)[0]['long'])
                        except:
                            print("zip not found, ",zip_hold)

                    else:
                        zip_var_center=zip_centers[zip_var]

                    try:
                        dist=haversine(zip_hold_center,zip_var_center,miles=True)
                    except:
                        dist=np.nan

                    dist_list=dist_list+[dist]
            df=pd.DataFrame({"TA":ta,"dist_min":min(dist_list),"dist_max":max(dist_list),"dist_median":np.median(dist_list),"All_dist":[dist_list]},index=[counter_k])
            counter_k+=1
            output_distance_zip_in_TA=output_distance_zip_in_TA.append(df)
        else:
            df=pd.DataFrame({"TA":ta,"dist_min":0,"dist_max":0,"dist_median":0,"All_dist":"single_zip"},index=[counter_k])
            output_distance_zip_in_TA=output_distance_zip_in_TA.append(df)

    zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
    zip_DMA=zip_DMA.iloc[:,[0,2]]
    zip_DMA.columns=['zip','DMA']

    df_city_state_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
    del df_city_state_in_TA['index']

    def city_of_zip(x):
        try:
            city=zipcodes.matching(x)[0]['city']
        except:
            city=np.nan
        return city

    def state_of_zip(x):
        try:
            state=zipcodes.matching(x)[0]['state']
        except:
            state=np.nan
        return state

    df_city_state_in_TA['city']=df_city_state_in_TA['zip'].apply(lambda x: city_of_zip(x))
    df_city_state_in_TA['state']=df_city_state_in_TA['zip'].apply(lambda x: state_of_zip(x))
    
    manually_city_dict_of_zip={"29486":"Summerville"}
    manually_state_dict_of_zip={"29486":"SC"}
    df_city_state_in_TA['city'][df_city_state_in_TA['zip']=='29486']=manually_city_dict_of_zip['29486']
    df_city_state_in_TA['state'][df_city_state_in_TA['zip']=='29486']=manually_state_dict_of_zip['29486']

    removed_2_zips_df=df_city_state_in_TA[pd.isnull(df_city_state_in_TA['city'])]
    df_city_state_in_TA=df_city_state_in_TA[~pd.isnull(df_city_state_in_TA['city'])]
    
    df_city_state_in_TA['city']=df_city_state_in_TA['city']+" ("+df_city_state_in_TA['state']+")"

    df_DMA_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
    del df_DMA_in_TA['index']
    df_DMA_in_TA=pd.merge(df_DMA_in_TA,zip_DMA,on="zip",how="left")

    df_city_TA_list=df_city_state_in_TA.groupby(['TA'])['city'].apply(set).to_frame().reset_index()
    df_city_TA_list=df_city_TA_list.rename(columns={"city":"city_list"})
    df_state_TA_list=df_city_state_in_TA.groupby(['TA'])['state'].apply(set).to_frame().reset_index()
    df_state_TA_list=df_state_TA_list.rename(columns={"state":"state_list"})
    df_DMA_TA_list=df_DMA_in_TA.groupby(['TA'])['DMA'].apply(set).to_frame().reset_index()
    df_DMA_TA_list=df_DMA_TA_list.rename(columns={"DMA":"DMA_list"})

    counter_k
    df_primary_city_state=pd.DataFrame()
    df_primary_DMA=pd.DataFrame()
    
    df_city_state_in_TA
    for ta,group in df_city_state_in_TA.groupby(['TA']):
        df_city=group.groupby(['city'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
        del df_city['index']
        primary_city=df_city['city'][0]

        df_state=group.groupby(['state'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
        del df_state['index']
        primary_state=df_state['state'][0]

        df=pd.DataFrame({"TA":ta,"Primary_City":primary_city,"Primary_State":primary_state},index=[counter_k])
        counter_k+=1
        df_primary_city_state=df_primary_city_state.append(df)


    for ta,group in df_DMA_in_TA.groupby(['TA']):
        df_DMA=group.groupby(['DMA'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
        del df_DMA['index']
        primary_DMA=df_DMA['DMA'][0]

        df=pd.DataFrame({"TA":ta,"Primary_DMA":primary_DMA},index=[counter_k])
        counter_k+=1
        df_primary_DMA=df_primary_DMA.append(df)

    summary_by_TA=pd.merge(summary_by_TA,df_city_TA_list,on="TA",how="left")
    summary_by_TA=pd.merge(summary_by_TA,df_state_TA_list,on="TA",how="left")
    summary_by_TA=pd.merge(summary_by_TA,df_DMA_TA_list,on="TA",how="left")

    summary_by_TA=pd.merge(summary_by_TA,df_primary_city_state,on="TA",how="left")
    summary_by_TA=pd.merge(summary_by_TA,df_primary_DMA,on="TA",how="left")

    summary_by_TA=pd.merge(summary_by_TA,output_distance_zip_in_TA,on="TA",how="left")
    summary_by_TA['Old_TA']=summary_by_TA['TA'].apply(lambda x: int(x.split("_")[0]))
    summary_by_TA=summary_by_TA.sort_values(['Old_TA','TA'])

    summary_by_TA['ratio_max_media']=summary_by_TA['dist_max']/summary_by_TA['dist_median']

    summary_by_TA_to_revise=summary_by_TA[(summary_by_TA['ratio_max_media']>2) &\
                                          (summary_by_TA['store_count']>=2) &\
                                         (summary_by_TA['dist_max']>12)]
    summary_by_TA_to_keep=summary_by_TA[(summary_by_TA['ratio_max_media']<=2) |\
                                          (summary_by_TA['store_count']<2) |\
                                         (summary_by_TA['dist_max']<=12)]

In [37]:
summary_by_TA_output=pd.DataFrame()
summary_by_TA=summary_by_TA.sort_values(['Old_TA'])
for old_ta,group in summary_by_TA.groupby(['Old_TA']):
    if len(group)>1:
        group['TA']=[str(old_ta).zfill(3)+"_"+str(x+1) for x in range(len(group))]
    else:
        group['TA']=str(old_ta).zfill(3)
    summary_by_TA_output=summary_by_TA_output.append(group)
ta_store_list_dic=summary_by_TA_output.set_index(['TA']).to_dict()['store_list']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [38]:
store_to_new_ta_dict={}
for new_ta in ta_store_list_dic.keys():
    length=len(ta_store_list_dic[new_ta])
    df=pd.DataFrame({"store":ta_store_list_dic[new_ta],"new_ta":[new_ta]*length},index=[x for x in range(length)])
    df_dict=df.set_index(['store']).to_dict()['new_ta']
    store_to_new_ta_dict.update(df_dict)
len(store_to_new_ta_dict)    
data['TA']=data['storenumber'].apply(lambda x: store_to_new_ta_dict[x])
df_TA_zips['TA']=df_TA_zips['store'].apply(lambda x: store_to_new_ta_dict[x])

summary_by_store_count=summary_by_TA_output.groupby(['store_count'])['TA'].count().to_frame().reset_index().rename(columns={"TA":"TA_count"})
summary_by_store_list=summary_by_TA_output.groupby(['store_count'])['TA'].apply(list).to_frame().reset_index().rename(columns={"TA":"TA_list"})
summary_by_store_count=pd.merge(summary_by_store_count,summary_by_store_list,on="store_count",how="outer")

In [39]:
data_for_tableau=data[['storenumber','revised_store_zip','city','state','Latitude','Longitude','TA']]
for col in data_for_tableau.columns.tolist()[1:6]:
    data_for_tableau=data_for_tableau.rename(columns={col:"store_"+col})
data_for_tableau=data_for_tableau.append(df_TA_zips[['TA','zip']])
data_for_tableau['store_Latitude']=data_for_tableau['store_Latitude'].fillna(0)
data_for_tableau['store_Longitude']=data_for_tableau['store_Longitude'].fillna(0)

In [42]:
writer=pd.ExcelWriter("/home/jian/Projects/Smoothie_King/TA/SmoothieKing_TA_revised_3_miles_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
summary_by_TA_output.to_excel(writer,"summary_by_TA",index=False)
data['storenumber']=data['storenumber'].astype(int)
data=data.sort_values(['storenumber'])
data.to_excel(writer,"output_TA_by_store",index=False)
df_TA_zips.to_excel(writer,"zip_TA",index=False)
data_for_tableau.to_excel(writer,"data_for_tableau",index=False)
summary_by_store_count.to_excel(writer,"summary_by_store_count",index=False)
closed_stores.to_excel(writer,"closed_stores",index=False)
del overwrite_zip_stores['Bing_LatLng']
del overwrite_zip_stores['Bing_Address']
overwrite_zip_stores.to_excel(writer,"stores_zips_overwritten",index=False)
writer.save()